### Clonar repositório crawl4ai

In [ ]:
!git clone https://github.com/unclecode/crawl4ai.git --quiet

### Salva o histórico de merges das branches

In [18]:
%cd /content/crawl4ai
!git log --graph --oneline --all --decorate --merges > /content/historico_merges.txt

/content/crawl4ai


### Lê os principais arquivos com informações relacionadas às releases e as branches

In [19]:
changelog = ""
releases = ""
merges = ""
with open("/content/crawl4ai/CHANGELOG.md", "r") as f:
  changelog = f.read()
with open("/content/crawl4ai/.github/workflows/release.yml", "r") as f:
  releases = f.read()
with open("/content/historico_merges.txt", "r") as f:
  merges = f.read()

### Criação do prompt

In [20]:
system_msg = f"""Você é um Especialista em Engenharia de Software com foco em Gerenciamento de Configuração. Sua tarefa é analisar o repositório "Crawl4AI" para identificar sua Estratégia de Release e seu Fluxo de Trabalho (Branching Model).

PRINCIPAIS ESTRATÉGIAS DE WORKFLOW PARA PROCURAR:
- Gitflow: Presença de branches persistentes 'main' E 'develop'. Merges de features vão para 'develop'. Existem branches de 'release/v*'.
- GitHub Flow: Apenas a branch 'main' é persistente. Features branches são curtas e mergeadas direto na 'main'.
- Trunk-based: Commits frequentes e diretos na 'main' (ou via PRs curtíssimos). O histórico é quase uma linha única e reta.

PRINCIPAIS ESTRATÉGIAS DE RELEASE PARA PROCURAR:
- Rapid Release: Alta frequência de versões (tags v* constantes) baseadas em novas funcionalidades prontas.
- Release Train: Versões em datas fixas e previsíveis.
- LTS + Current: Coexistência de versões estáveis de longo suporte com versões de ponta.

VOCÊ RECEBERÁ DADOS DOS SEGUINTES ARQUIVOS:
- CHANGELOG.md: Registro histórico de versões e datas.
- release.yml: Configuração de automação do GitHub Actions.
- Histórico de Merges: O log visual (graph) da árvore de commits.

METODOLOGIA DE ANÁLISE:
1. Examine o 'release.yml': Tente identificar o que dispara o deploy.
2. Examine o 'CHANGELOG.md': Tente identificar o intervalo médio entre versões.
3. Examine o 'Histórico de Merges': Procure pela existência da branch 'develop' e branches com prefixo 'release/v*', 'feature/*' ou 'fix/* e verificar a persistência dessas branches'.

CONCLUSÃO:
Apresente sua conclusão baseada na predominância das evidências. Se houver elementos de mais de um modelo (ex: um fluxo Gitflow simplificado com ritmo de Rapid Release), descreva essa governança híbrida."""
user_msg = f"""
Aqui estão os arquivos a serem analisados:
CHANGELOG.md:
{"\n".join(changelog.splitlines()[-262:])}

release.yml:
{releases}

Histórico de Merges:
{merges}
"""

### Rodar o modelo usando Inference Provider do Hugging Face

In [21]:
from google.colab import userdata
import os
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

In [25]:
from openai import OpenAI

client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=os.environ["HF_TOKEN"],
)

completion = client.chat.completions.create(
    model="meta-llama/Llama-3.1-8B-Instruct",
    messages=[
        {
            "role": "system",
            "content": system_msg
        },
        {
            "role": "user",
            "content": user_msg
        }
    ],
    max_tokens=1500,
    temperature=0.2,
    top_p=0.9
)

with open("/content/resposta-llama-3_2-8B-Instruct.md", "w") as f:
  f.write(completion.choices[0].message.content)